In [34]:
from pathlib import Path
from uuid import uuid4
from hashlib import md5
from tempfile import NamedTemporaryFile
import re
from shutil import copy

import shapely.geometry as geo
import viewscad
import vsketch
from solid import *
from solid.utils import *  # Not required, but the utils module is useful


In [35]:
def hash_string(string: str):
    return md5(string.encode()).hexdigest()[:7]


In [36]:
r = viewscad.Renderer()


In [37]:
def vsketch_to_scad_2d(sketch: vsketch.Vsketch, output_folder: Path = Path("output/")):
    with NamedTemporaryFile(suffix=".svg") as tempsvg:
        tempsvg = NamedTemporaryFile(suffix=".svg")
        sketch.save(tempsvg.name)
        svg_text = Path(tempsvg.name).read_text()
        # vsketch dynamically generates a timestamp
        # rather than parse the svg as XML, let's just regex it out
        svg_text = re.sub(
            r"^.*\<dc\:date\>.*\<\/dc\:date\>$", "", svg_text, flags=re.MULTILINE
        )
        svg_hash = hash_string(svg_text)

    svg_path = output_folder / (svg_hash + ".svg")
    sketch.save(svg_path)

    SCALE = 1 / 0.2645833333
    SCAD_TEMPLATE = f"""
    module svg_{svg_hash}()
    {{
        scale([{SCALE}, {SCALE}, 0])
        {{
            import("{svg_path.resolve()}", dpi=96);
        }};
    }}
    """.strip()

    scadfile_temp = output_folder / (svg_hash + ".scad")
    scadfile_temp.write_text(SCAD_TEMPLATE)
    scadfile = import_scad(scadfile_temp)
    shape_2d = getattr(scadfile, f"svg_{svg_hash}")()
    return shape_2d


def svg_to_scad_2d(svg: Union[str, Path], output_folder: Path = Path("output/")):
    if isinstance(svg, str):
        svg = Path(svg)
    with NamedTemporaryFile(suffix=".svg") as tempsvg:
        tempsvg = NamedTemporaryFile(suffix=".svg")
        svg_text = svg.read_text()
        # vsketch dynamically generates a timestamp
        # rather than parse the svg as XML, let's just regex it out
        svg_text = re.sub(
            r"^.*\<dc\:date\>.*\<\/dc\:date\>$", "", svg_text, flags=re.MULTILINE
        )
        svg_hash = hash_string(svg_text)

    svg_path = output_folder / (svg_hash + ".svg")
    copy(svg, svg_path)

    SCALE = 1
    SCAD_TEMPLATE = f"""
    module svg_{svg_hash}()
    {{
        scale([{SCALE}, {SCALE}, 0])
        {{
            import("{svg_path.resolve()}", center=true);
        }};
    }}
    """.strip()

    scadfile_temp = output_folder / (svg_hash + ".scad")
    scadfile_temp.write_text(SCAD_TEMPLATE)
    scadfile = import_scad(scadfile_temp)
    shape_2d = getattr(scadfile, f"svg_{svg_hash}")()
    return shape_2d


In [48]:
import numpy as np

x = 18
y = 8
z = 6

scale_val = 1.5

issues = [3756, 1105, 132, 1971, 1508, 11005, 125, 485, 1969, 9606, 2011, 3339, 6990, 285, 3082, 15, 3618, 641, 507, 253, 5389, 4511, 3356, 2852, 9425, 4119, 4259, 2668, 7264, 4695, 4083, 579, 429, 7687, 4758, 232]

all_shapes = []
for i, issue in enumerate(issues):
    x_trans = (i % 3) * ((x + 4)* scale_val) 
    y_trans = (i // 3) * ((y + 4)* scale_val) 
    print((i // 3), (i % 12))
    base = linear_extrude(z)(offset(1, segments=100)(square([x - 1, y - 1], center=True)))

    inset_2d = svg_to_scad_2d(
        "/Users/peter/projects/py-open-scad/explosionopoly/closed-issue-2.svg"
    )
    inset_extruded = scale([0.8, 0.8, 1])(translate([-8,0,z/2])(linear_extrude(z/2)(inset_2d)))

    number_text = translate([2.5,0,z/2])(linear_extrude(z/2)(text(text=f"{issue}", font="Inter:style=Bold", size=3, halign="center", valign="center", segments=100)))

    result = scale([scale_val, scale_val, 1])(base - (inset_extruded + number_text))
    result_translated = translate([x_trans, y_trans, 0])(result)
    all_shapes.append(result_translated)

final = union()(*all_shapes)
# r.render(result)
scad_render_to_file(all_shapes, f"all_issues.scad")


0 0
0 1
0 2
1 3
1 4
1 5
2 6
2 7
2 8
3 9
3 10
3 11
4 0
4 1
4 2
5 3
5 4
5 5
6 6
6 7
6 8
7 9
7 10
7 11
8 0
8 1
8 2
9 3
9 4
9 5
10 6
10 7
10 8
11 9
11 10
11 11


AttributeError: 'list' object has no attribute 'children'

In [56]:
import numpy as np

x = 18
y = 8
z = 6

scale_val = 2

issues = [6862, 3413, 5907, 1501, 1502, 5477, 3153, 3079, 4828, 3285, 2983, 1385]
all_shapes = []
for i, issue in enumerate(issues):
    x_trans = (i % 4) * ((x + 4)* scale_val) 
    y_trans = (i // 4) * ((y + 4)* scale_val) 
    print((i // 3), (i % 12))
    base = linear_extrude(z)(offset(1, segments=50)(square([x - 1, y - 1], center=True)))

    inset_2d = svg_to_scad_2d(
        "/Users/peter/projects/py-open-scad/explosionopoly/merged-pr-2.svg"
    )
    inset_extruded = scale([0.8, 0.8, 1])(translate([-7,0,z/2])(linear_extrude(z/2)(inset_2d)))

    number_text = translate([3,0,z/2])(linear_extrude(z/2)(text(text=f"{issue}", font="Inter:style=Bold", size=3, halign="center", valign="center", segments=50)))

    result = scale([scale_val, scale_val, scale_val])(base - (inset_extruded + number_text))
    result_translated = translate([x_trans, y_trans, 0])(result)
    all_shapes.append(result_translated)

final = union()(*all_shapes)
# r.render(result)
scad_render_to_file(final, f"all_prs.scad")


0 0
0 1
0 2
1 3
1 4
1 5
2 6
2 7
2 8
3 9
3 10
3 11


'/Users/peter/projects/py-open-scad/explosionopoly/all_prs.scad'